# This file will evaluate students' homeworks using AI assistant

## 1. Introduce AI tool first

In [1]:
# If use ollama, no API_KEY needed
%load_ext jupyter_ai_magics
%config AiMagics.default_language_model = 'ollama:phi4'

In [ ]:
# If use Gemini, have to load .env file to get API_KEY
%load_ext jupyter_ai_magics
%config AiMagics.default_language_model = 'gemini:gemini-2.0-flash-thinking-exp'

from dotenv import load_dotenv

load_dotenv()

## 2. Use AI to evaluate the given code 

In [151]:
import nbformat, os
from nbclient import NotebookClient

# 超参数设置
# keyword: 在作业中用于识别待批阅单元格的关键词，要与作业文件中相匹配
# root_dir: 作业文件夹位置
keyword = '----Answer----'
root_dir = './homework'

class HomeworkManager:
    def __init__(self):
        self.key_word = ''
        self.root_dir = ''
        print("Please provide key_word and root_dir first!")
        
    def get_all_homework(self):
        all_homeworks = []
        for dir_path, _, file_names in os.walk(self.root_dir):
            for file_name in file_names:
                file_path = os.path.join(dir_path, file_name)
                all_homeworks.append(file_path)
        return all_homeworks
    
    def get_cell_content(self, file_path):
        with open(file_path) as f:
            nb = nbformat.read(f, as_version=4)
        review_cell = [cell for cell in nb.cells if self.key_word.lower() in cell.source.lower()]
        if len(review_cell) == 0:
            raise IndexError("There's no answer in the notebook.")
        else:
            return review_cell

In [ ]:
import nbformat, os
from nbclient import NotebookClient

keyword = '----Answer----'
root_dir = './homework'

def get_cell_content(notebook_path, key_word):
    with open(notebook_path,) as f:
        nb = nbformat.read(f, as_version=4)
    review_cell = [cell for cell in nb.cells if key_word.lower() in cell.source.lower()]
    if len(review_cell) == 0:
        raise IndexError("There's no anwser in the notebook.")
    else:
        return review_cell

def get_all_homework(root_dir):
    all_homework = []
    for dir_path, _, filenames in os.walk(root_dir):
        for filename in filenames:
            file_path = os.path.join(dir_path, filename)
            all_homework.append(file_path)
            print(file_path)

    return all_homework

all_homeworks = get_all_homework(root_dir)


In [154]:
import time
from IPython.core.getipython import get_ipython

class NotebookManager:
    def __init__(self):
        self.shell = get_ipython()

    def create_code_cell(self, content, cell_type='code'):
        """Create new code cell"""
        payload = dict(
            source = 'set_next_input',
            text = content,
            replace = False
        )
        self.shell.payload_manager.write_payload(payload)

    def create_markdown_cell(self, content, cell_type='markdown'):
        """Create new markdown cell"""
        payload = dict(
            source = 'set_next_input',
            text = content,
            replace = False
        )
        self.shell.payload_manager.write_payload(payload)

    def run_cell(self, code):
        result = self.shell.run_cell(code)
        return result
    
    def create_and_run(self, content):
        self.create_code_cell(content)
        time.sleep(0.5)
        return self.run_cell(content)

In [ ]:
hws = HomeworkManager()
hws.key_word = keyword
hws.root_dir = root_dir

homeworks = hws.get_all_homework()

nb = NotebookManager()

for homework in homeworks:
    file_name = os.path.basename(homework)
    stu_name = file_name.split('_')[0]
    code_seg = hws.get_cell_content(homework)[0].source
    # print(hw_dict['name'])
    prompt = "%%ai\n"
    prompt += "{stu_name} is a beginner in Python coding. Please review his(her) code shown in {code_seg}, rated them in 5-score system and give some advice from the perspective of {stu_name}'s teacher, omit advanced coding skills."
    prompt += "Use Chinese to format your answer."

    nb.create_and_run(prompt)
